# Title
[]()

In [9]:
# from pandas import json_normalize  
import pandas as pd
import sys
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\custom_python")
sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\portfolio-projects\online-PT-social-media-NLP\src")
from silvhua import *
from datetime import datetime
# import json
# from plotly.subplots import make_subplots
# import requests

In [10]:
# set the option to wrap text within cells
pd.set_option('display.max_colwidth', None)
# pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [11]:
# from pypdf import PdfReader
# from openai.embeddings_utils import get_embedding, cosine_similarity
import openai
import os
import re

In [12]:

sys.path.append(r"C:\Users\silvh\OneDrive\lighthouse\Ginkgo coding\content-summarization\src")
from summarization import Chatbot, reply, batch_reply
from article_processing import create_text_dict

## Create relevance_prompts_df

In [20]:
from itertools import product
prompts_dict = dict()
experiment_num = 1
relevance_prompts_dict = dict()

prep_step = [
    # "Take the key points to",
    "Take the key points and numerical descriptors to",
    # ""
]

task_part1 = [
    # "Summarize the article in under 300 characters",
    "Summarize for a LinkedIn post",
    # "Summarize for a tweet",
    # "Summarize in an engaging way",
    "Describe the interesting points to your coworker at the water cooler"
    # "Summarize the article for a Tiktok post"
]
audience = [
    # "lay audience",
    # "",
    "seniors",
    "people who enjoy sports"
]
task_part2 = [
    "",
    "Use terms a 12-year-old can understand.",
    # "Assume your audience has no science background."
    # "Include the most interesting findings.",
    # "Include the key take-aways for the reader.",
    # "Include the implications of the article."
]

task_part3 =[
    "Add 1-2 sentences to make this relevant for"
    # "Add 1-2 sentences to make this relevant for older adults."
    # "Once you are done, add 1-2 sentences to make this relevant for older adults.",
    ""
]

# prompts_df = pd.DataFrame(product(prep_step, task_part1, task_part2, task_part3, audience), 
#     columns=['prep_step', 'task part 1', 'task part 2', 'task part 3', 'audience'])

prompts_df = pd.DataFrame(product(prep_step, task_part1), 
    columns=['prep_step', 'task part 1'])

relevance_prompts_df = pd.DataFrame(product(task_part3, audience), 
    columns=['task part 3', 'audience'])



prompts_df['prompt'] = prompts_df.apply(
    lambda row: f"{row['prep_step']} {row['task part 1']}.", 
    axis=1)
# prompts_df['simplify'] = prompts_df.apply(
#     lambda row: f" {row['task part 2'] if row['task part 2'] else ''}", 
#     axis=1)
relevance_prompts_df['relevance'] = relevance_prompts_df.apply(
    lambda row: f" {row['task part 3']} {row['audience']} " if row['audience'] else '', 
    axis=1) 
relevance_prompts_dict[experiment_num] = relevance_prompts_df
relevance_prompts_df

,task part 3,audience,relevance
0,Add 1-2 sentences to make this relevant for,seniors,Add 1-2 sentences to make this relevant for seniors
1,Add 1-2 sentences to make this relevant for,people who enjoy sports,Add 1-2 sentences to make this relevant for people who enjoy sports


In [21]:
prompts_df

,prep_step,task part 1,prompt
0,Take the key points and numerical descriptors to,Summarize for a LinkedIn post,Take the key points and numerical descriptors to Summarize for a LinkedIn post.
1,Take the key points and numerical descriptors to,Describe the interesting points to your coworker at the water cooler,Take the key points and numerical descriptors to Describe the interesting points to your coworker at the water cooler.


# Import text files

In [5]:
filename = '../text/Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men.txt'
encoding='ISO-8859-1'
with open(filename, 'r', encoding=encoding) as file:
    text1 = file.read()
text1

'Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men\n\nDiscussion\nResults of this study indicated no differences in the recovery response between YA and MA for any of the performance measures, nor in subjective levels of muscle pain or soreness. Furthermore, no between-group differences were observed in the inflammatory or\xa0muscle damage\xa0response to the exercise protocol. To the best of our knowledge, this is the first study to examine differences in the recovery response from high-volume resistance exercise between recreationally trained young and middle-aged adults. Our results comparing BL performance measures seem to be in agreement with other investigations, demonstrating differences in strength measures between young (23\x9629 years) and older adults (60\x9665 years) (7,15,24), and between middle-aged (41\x9642 years) and older adults (70\x9672 years) (17,18). It does seem that the decline in strength may develop during middle ag

# Create text_dict

In [7]:
import os

folder_path = '../text/2023-04-13'
encoding='ISO-8859-1'
all_files = []

for filename in os.listdir(folder_path):
    with open(os.path.join(folder_path, filename), 'r', encoding=encoding) as f:
        all_files.append(f.read())

all_files

['Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men\n\nDiscussion\nResults of this study indicated no differences in the recovery response between YA and MA for any of the performance measures, nor in subjective levels of muscle pain or soreness. Furthermore, no between-group differences were observed in the inflammatory or\xa0muscle damage\xa0response to the exercise protocol. To the best of our knowledge, this is the first study to examine differences in the recovery response from high-volume resistance exercise between recreationally trained young and middle-aged adults. Our results comparing BL performance measures seem to be in agreement with other investigations, demonstrating differences in strength measures between young (23\x9629 years) and older adults (60\x9665 years) (7,15,24), and between middle-aged (41\x9642 years) and older adults (70\x9672 years) (17,18). It does seem that the decline in strength may develop during middle a

In [13]:
text_dict = create_text_dict(all_files)
text_dict

{1: 'Comparisons in the Recovery Response From Resistance Exercise Between Young and Middle-Aged Men\n\nDiscussion\nResults of this study indicated no differences in the recovery response between YA and MA for any of the performance measures, nor in subjective levels of muscle pain or soreness. Furthermore, no between-group differences were observed in the inflammatory or\xa0muscle damage\xa0response to the exercise protocol. To the best of our knowledge, this is the first study to examine differences in the recovery response from high-volume resistance exercise between recreationally trained young and middle-aged adults. Our results comparing BL performance measures seem to be in agreement with other investigations, demonstrating differences in strength measures between young (23\x9629 years) and older adults (60\x9665 years) (7,15,24), and between middle-aged (41\x9642 years) and older adults (70\x9672 years) (17,18). It does seem that the decline in strength may develop during middl

# *End of Page*